## 빅데이터 실습

### 다나와 무선청소기 상품분석

#### 데이터 수집

##### 셀레니움 사용

In [1]:
## 필수 라이브러리 사용등록
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib import parse # url encode
from tqdm import tqdm
import pandas as pd
import time

In [2]:
## 웹드라이버로 크롬 오픈
driver = webdriver.Chrome()
url = r'https://search.danawa.com/dsearch.php?query=무선청소기&page=1&limit=120&sort=saveDESC&list=list'
driver.get(url)
time.sleep(5.0)

In [3]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
soup

<html class="js no-touch" lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head>
<title>무선청소기 : 다나와 통합검색</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="무선청소기 통합검색 : 다나와 통합검색" name="title"/>
<meta content="'무선청소기'의 다나와 통합검색 결과입니다." name="description"/>
<meta content="무선청소기, 다나와, 최저가, 가격비교, 검색, 통합검색, 차이슨 무선청소기, 디베아 무선청소기, 삼성 무선청소기, 일렉트로룩스 무선청소기, 샤크 무선청소기, 다이슨 무선청소기, 테팔무선청소기, 샤오미 무선청소기, lg 무선청소기, 유선청소기" name="keywords"/>
<meta content="무선청소기 통합검색 : 다나와 통합검색" property="og:title"/>
<meta content="'무선청소기'의 다나와 통합검색 결과입니다." property="og:description"/>
<meta content="https://img.danawa.com/new/tour/img/logo/sns_danawa.jpg" property="og:image"/>
<meta content="width=1280" name="viewport"/>
<link href="//img.danawa.com/new/danawa_main/v1/img/danawa_favicon.ico" rel="shortcut icon"/>
<link href="//static.danawa.com/css/common.css?v=2402280851" rel="stylesheet" type="text/css"/>
<link hre

In [4]:
prodItems = soup.select('ul.product_list > li.prod_item')

In [5]:
len(prodItems)

122

In [6]:
prodItems[0].select('div.spec_list')[0].text.strip().replace('\t','')

'핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 250W / 소비전력: 620W / 2023년형 / [구성] 거치대: 올인원타워 / 먼지비움 / 충전 / UVC LED / 액서서리수납 / 스탠드거치 / 먼지비움시간: 30초 / 브러쉬: 와이드바닥 / 물걸레: 스팀, 고온, 일반 / 솔형 / 틈새 / 먼지봉투: 2.5L / [배터리] 사용시간: 30분(최대) / 충전시간: 4시간 / 분리형(2개) / 리튬이온 / 25.2V / [청소] 싸이클론흡입 / LED라이트 / 자동물공급 / 스마트인버터모터 / [부가] 스마트폰제어 / 대화형알림창 / 워셔블헤파필터 / 5단계여과 / 색상: 카밍베이지 / 무게: 2.47kg / 크기(가로x세로x깊이): 300x1120x245mm'

In [7]:
prodItems[0].select('input')[4].get('value')

'81'

In [13]:
# 만약에 수가 아닌 문자열 들어왔을때는 False
prodItems[0].select('input')[1].get('value').isdecimal()

True

#### 다나와 무선청소기 웹크롤링
- 가격외에는 안들어오도록 변환

In [14]:
## 검색어, 페이지를 변경하면서 URL 생성함수
def getSearchPageUrl(keyword, page):
    ecKeyword = parse.quote(keyword)
    url = f'https://search.danawa.com/dsearch.php?query={ecKeyword}&originalQuery={ecKeyword}&previousKeyword={ecKeyword}&checkedInfo=N&volumeType=allvs&' + \
          f'page={page}&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&' + \
           'recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&' + \
           'quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'
    return url

In [16]:
## 상품정보 추출하는 함수
def getProdItems(prodItems):
    prodData = []

    for prodItem in prodItems:
        try:
            prodName = prodItem.select('p.prod_name > a')[0].text.strip()  # 상품명 가져오기
            specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t', '') # 상품 스펙목록 가져오기
            
            if prodItem.select('input')[1].get('value').isdecimal() == True:
              price = prodItem.select('input')[1].get('value')  
            else:
              price = 0 # 문자열 들어온 것 막기

            prodData.append([prodName, specList, price])
        except:
            pass
    
    return prodData